In [1]:
import pandas as pd
import numpy as np
import consultas as cst

In [2]:
pd.options.display.max_columns = False

In [ ]:
#Filtro del ending
query=f'''
SELECT * 
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   A.PERIODO_PROCESO_CODIGO=202210
        AND A.SERVICIO='Prepaid'
        AND A.MOVIMIENTO_TIPO_NOMBRE='CHURN PORTACION'
'''

In [ ]:
#PRIMERA PARTE pa construir la temporal de consultores
query=f'''
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        B.CONSULTOR_NOMBRE_NIVEL_8

FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B 
ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE   A.PERIODO_PROCESO_CODIGO=202210
        AND A.SERVICIO='Prepaid'
'''

In [3]:
#QUERY Que agrupa los movimientos tipo nombre del ending con sus consultores, se debe pivotear 
query1=f'''
SELECT  CONSULTOR_NOMBRE_NIVEL_8,
        MOVIMIENTO_TIPO_NOMBRE,
        COUNT(SUBSCRIBER_ID) AS TOTAL_CONSULTOR
FROM(        

SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        B.CONSULTOR_NOMBRE_NIVEL_8
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B 
ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE   A.PERIODO_PROCESO_CODIGO=202210
        AND A.SERVICIO='Prepaid'
) C 
GROUP BY CONSULTOR_NOMBRE_NIVEL_8,MOVIMIENTO_TIPO_NOMBRE
'''
consultores=cst.consulta(query1)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [15]:
#tabla historica de los dealers
query=f'''
SELECT /*+PARALLEL(8)*/
    B.CONSULTOR_NOMBRE_NIVEL_8,
    SUM(CASE
        WHEN A.MOVIMIENTO_NOMBRE = 'ACTIVACION' THEN 1 ELSE 0
    END) AS GROSS_TOTAL,
    SUM(CASE
        WHEN A.MOVIMIENTO_NOMBRE = 'DESCONEXION' THEN 1 ELSE 0
    END) AS CHURN_TOTAL,
    SUM(CASE
        WHEN A.MOVIMIENTO_TIPO_NOMBRE = 'CHURN PORTACION' THEN 1 ELSE 0
    END) AS CHURN_PORTACION,
    SUM(CASE
        WHEN A.MOVIMIENTO_TIPO_NOMBRE = 'CHURN INACTIVIDAD' THEN 1 ELSE 0
    END) AS CHURN_INACTIVIDAD,
    SUM(CASE
        WHEN (A.MOVIMIENTO_TIPO_NOMBRE = 'PERMANECE' AND A.PERIODO_PROCESO_CODIGO = 202210) THEN 1 ELSE 0
    END) AS PERMANECE_ACT,
    SUM(CASE
        WHEN (A.MOVIMIENTO_NOMBRE = 'ACTIVACION' AND A.PERIODO_PROCESO_CODIGO = 202210) THEN 1 ELSE 0
    END) AS GROSS_ACT,
    SUM(CASE
        WHEN (A.MOVIMIENTO_TIPO_NOMBRE = 'DESCONEXION' AND A.PERIODO_PROCESO_CODIGO = 202210) THEN 1 ELSE 0
    END) AS CHURN_ACT
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B ON A.CONSULTOR_DK = B.CONSULTOR_DK
WHERE A.SERVICIO='Prepaid'
GROUP BY
    B.CONSULTOR_NOMBRE_NIVEL_8
'''
consultores_historico=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [16]:
consultores_historico.sort_values('GROSS_TOTAL',ascending=False)

,CONSULTOR_NOMBRE_NIVEL_8,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT
134,Tech Red Sas,349408,190372,33616,148745,131343,26385,0
0,Aloha Comunicaciones Sas,277277,203342,70229,129965,63205,11308,0
491,Beyotta Sas Beyotta Sas,232290,184042,74293,103983,46366,1877,0
22,One Movil Sas,204705,118091,19683,91981,76048,9716,0
224,Ventas 577 Ltda,188449,140791,65272,70225,43989,3514,0
...,...,...,...,...,...,...,...,...
169,Conectamos de Colombia Sas,1,1,0,0,1,0,0
60,LogycTour Partner,1,0,0,0,1,0,0
93,Movilfon Sas,1,0,0,0,1,0,0
223,Stiven Morales Arenas,1,0,0,0,0,0,0


In [5]:
consultores_copia=consultores.copy()
consultores_copia.head()

,CONSULTOR_NOMBRE_NIVEL_8,MOVIMIENTO_TIPO_NOMBRE,TOTAL_CONSULTOR
0,Fuerza Comercial Multiservicios Sas,GROSS ADDS NUEVA,20819
1,-2,GROSS ADDS NUEVA,854
2,One Movil Sas,GROSS ADDS NUEVA,9664
3,Sebastian Garcia Restrepo,GROSS ADDS NUEVA,41
4,Jhon Fredy Castellanos Lopez,GROSS ADDS PORTACION,9


In [6]:
consultor_totales=consultores_copia.groupby('CONSULTOR_NOMBRE_NIVEL_8').sum()

In [7]:
consultor_movimiento=consultores_copia.pivot_table(index='CONSULTOR_NOMBRE_NIVEL_8',columns='MOVIMIENTO_TIPO_NOMBRE',values='TOTAL_CONSULTOR')
consultor_movimiento=consultor_movimiento.fillna(0)
consultor_movimiento=consultor_movimiento.merge(consultor_totales,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8',how='inner')
consultor_movimiento.head()

,ACTIVACION NO GROSS,CHURN FRAUDE,CHURN INACTIVIDAD,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,DESCONEXION NO GROSS,GROSS ADDS MIGRACION,GROSS ADDS NUEVA,GROSS ADDS PORTACION,PERMANECE,PERMANECE NO GROSS,TOTAL_CONSULTOR
CONSULTOR_NOMBRE_NIVEL_8,,,,,,,,,,,,,
-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,13
-1,0.0,0.0,214.0,3.0,11.0,1.0,0.0,0.0,0.0,0.0,1126.0,0.0,1355
-2,11999.0,0.0,939.0,72.0,103.0,40.0,2011.0,3.0,854.0,52.0,10084.0,21810.0,47967
12g Comunicaciones Sas,0.0,0.0,742.0,38.0,79.0,12.0,0.0,0.0,100.0,2.0,8137.0,0.0,9110
5gcomunicaciones Sas,0.0,0.0,1432.0,72.0,58.0,28.0,1.0,0.0,0.0,0.0,12315.0,0.0,13906


In [27]:
consultor_movimiento[['CHURN FRAUDE','CHURN INACTIVIDAD']].reset_index()

,CONSULTOR_NOMBRE_NIVEL_8,CHURN FRAUDE,CHURN INACTIVIDAD
0,-,0.0,0.0
1,-1,0.0,214.0
2,-2,0.0,939.0
3,12g Comunicaciones Sas,0.0,742.0
4,5gcomunicaciones Sas,0.0,1432.0
...,...,...,...
684,jennyfer ramirez,0.0,0.0
685,jhorman alonso toro arias,0.0,1.0
686,juan gabriel gomez zapata,0.0,7.0
687,julio miguel acosta tapia,0.0,11.0


In [8]:
consultor_movimiento["GROSS_TOTAL"]=consultor_movimiento["GROSS ADDS MIGRACION"]+consultor_movimiento["GROSS ADDS NUEVA"]+consultor_movimiento["GROSS ADDS PORTACION"]
consultor_movimiento["CHURN_TOTAL"]=consultor_movimiento["CHURN FRAUDE"]+consultor_movimiento["CHURN INACTIVIDAD"]+consultor_movimiento["CHURN MIGRACION"]+consultor_movimiento["CHURN PORTACION"]+consultor_movimiento["CHURN VOLUNTARIO"]


In [9]:
def division(x,y):
    try :
        return (x/y)*100
    except:
        return 0    

In [10]:
consultor_movimiento["RATIO_CHURN"]=consultor_movimiento.apply(lambda x:division(x["CHURN_TOTAL"],x["TOTAL_CONSULTOR"]),axis=1)
consultor_movimiento["RATIO_CHURN_PORTACION"]=consultor_movimiento.apply(lambda x:division(x["CHURN PORTACION"],x["TOTAL_CONSULTOR"]),axis=1)
consultor_movimiento["RATIO_CHURN_INACTIVIDAD"]=consultor_movimiento.apply(lambda x:division(x["CHURN INACTIVIDAD"],x["TOTAL_CONSULTOR"]),axis=1)
consultor_movimiento.sort_values('GROSS_TOTAL',ascending=False)

,ACTIVACION NO GROSS,CHURN FRAUDE,CHURN INACTIVIDAD,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,DESCONEXION NO GROSS,GROSS ADDS MIGRACION,GROSS ADDS NUEVA,GROSS ADDS PORTACION,PERMANECE,PERMANECE NO GROSS,TOTAL_CONSULTOR,GROSS_TOTAL,CHURN_TOTAL,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
CONSULTOR_NOMBRE_NIVEL_8,,,,,,,,,,,,,,,,,,
Tech Red Sas,0.0,0.0,12218.0,440.0,1377.0,152.0,61.0,0.0,26298.0,87.0,131343.0,7.0,171983,26385.0,14187.0,8.249071,0.800661,7.104191
Fuerza Comercial Multiservicios Sas,0.0,0.0,12705.0,78.0,1319.0,17.0,0.0,0.0,20819.0,58.0,38199.0,0.0,73195,20877.0,14119.0,19.289569,1.802036,17.357743
Inversiones Bitec S A S,0.0,0.0,8160.0,310.0,3158.0,111.0,0.0,0.0,13362.0,27.0,62092.0,0.0,87220,13389.0,11739.0,13.459069,3.620729,9.355652
Conexxion Outsourcing sas,0.0,0.0,7001.0,12.0,16.0,2.0,0.0,0.0,13029.0,10.0,16207.0,0.0,36277,13039.0,7031.0,19.381426,0.044105,19.298729
Aloha Comunicaciones Sas,0.0,0.0,12431.0,192.0,14415.0,75.0,39.0,1.0,11284.0,23.0,63205.0,25.0,101690,11308.0,27113.0,26.662405,14.175435,12.224408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Luis Heiver Arias Lopez,0.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,129.0,0.0,134,0.0,5.0,3.731343,1.492537,1.492537
Cindy Paola Arango,0.0,0.0,20.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,235.0,0.0,257,0.0,22.0,8.560311,0.389105,7.782101
Luis Ernesto Muñoz Ballesteros,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,8,0.0,0.0,0.000000,0.000000,0.000000


In [34]:
base=consultor_movimiento.sort_values('TOTAL_CONSULTOR',ascending=False)
base.head()

,ACTIVACION NO GROSS,CHURN FRAUDE,CHURN INACTIVIDAD,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,DESCONEXION NO GROSS,GROSS ADDS MIGRACION,GROSS ADDS NUEVA,GROSS ADDS PORTACION,PERMANECE,PERMANECE NO GROSS,TOTAL_CONSULTOR,GROSS_TOTAL,CHURN_TOTAL,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
CONSULTOR_NOMBRE_NIVEL_8,,,,,,,,,,,,,,,,,,
Tech Red Sas,0.0,0.0,12218.0,440.0,1377.0,152.0,61.0,0.0,26298.0,87.0,131343.0,7.0,171983,26385.0,14187.0,8.249071,0.800661,7.104191
Aloha Comunicaciones Sas,0.0,0.0,12431.0,192.0,14415.0,75.0,39.0,1.0,11284.0,23.0,63205.0,25.0,101690,11308.0,27113.0,26.662405,14.175435,12.224408
One Movil Sas,0.0,0.0,6738.0,476.0,2078.0,128.0,8.0,0.0,9664.0,52.0,76048.0,23.0,95215,9716.0,9420.0,9.893399,2.182429,7.076616
Inversiones Bitec S A S,0.0,0.0,8160.0,310.0,3158.0,111.0,0.0,0.0,13362.0,27.0,62092.0,0.0,87220,13389.0,11739.0,13.459069,3.620729,9.355652
Dsurtiendo Sas,0.0,0.0,5998.0,301.0,573.0,100.0,1.0,0.0,10817.0,209.0,57130.0,0.0,75129,11026.0,6972.0,9.280038,0.762688,7.983602


In [35]:
permanece=consultor_movimiento.sort_values('PERMANECE',ascending=False)
permanece.head()

,ACTIVACION NO GROSS,CHURN FRAUDE,CHURN INACTIVIDAD,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,DESCONEXION NO GROSS,GROSS ADDS MIGRACION,GROSS ADDS NUEVA,GROSS ADDS PORTACION,PERMANECE,PERMANECE NO GROSS,TOTAL_CONSULTOR,GROSS_TOTAL,CHURN_TOTAL,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
CONSULTOR_NOMBRE_NIVEL_8,,,,,,,,,,,,,,,,,,
Tech Red Sas,0.0,0.0,12218.0,440.0,1377.0,152.0,61.0,0.0,26298.0,87.0,131343.0,7.0,171983,26385.0,14187.0,8.249071,0.800661,7.104191
One Movil Sas,0.0,0.0,6738.0,476.0,2078.0,128.0,8.0,0.0,9664.0,52.0,76048.0,23.0,95215,9716.0,9420.0,9.893399,2.182429,7.076616
Aloha Comunicaciones Sas,0.0,0.0,12431.0,192.0,14415.0,75.0,39.0,1.0,11284.0,23.0,63205.0,25.0,101690,11308.0,27113.0,26.662405,14.175435,12.224408
Inversiones Bitec S A S,0.0,0.0,8160.0,310.0,3158.0,111.0,0.0,0.0,13362.0,27.0,62092.0,0.0,87220,13389.0,11739.0,13.459069,3.620729,9.355652
Dsurtiendo Sas,0.0,0.0,5998.0,301.0,573.0,100.0,1.0,0.0,10817.0,209.0,57130.0,0.0,75129,11026.0,6972.0,9.280038,0.762688,7.983602


In [36]:
churn_total=consultor_movimiento.sort_values('CHURN_TOTAL',ascending=False)
churn_total.head()

,ACTIVACION NO GROSS,CHURN FRAUDE,CHURN INACTIVIDAD,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,DESCONEXION NO GROSS,GROSS ADDS MIGRACION,GROSS ADDS NUEVA,GROSS ADDS PORTACION,PERMANECE,PERMANECE NO GROSS,TOTAL_CONSULTOR,GROSS_TOTAL,CHURN_TOTAL,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
CONSULTOR_NOMBRE_NIVEL_8,,,,,,,,,,,,,,,,,,
Aloha Comunicaciones Sas,0.0,0.0,12431.0,192.0,14415.0,75.0,39.0,1.0,11284.0,23.0,63205.0,25.0,101690,11308.0,27113.0,26.662405,14.175435,12.224408
Tech Red Sas,0.0,0.0,12218.0,440.0,1377.0,152.0,61.0,0.0,26298.0,87.0,131343.0,7.0,171983,26385.0,14187.0,8.249071,0.800661,7.104191
Fuerza Comercial Multiservicios Sas,0.0,0.0,12705.0,78.0,1319.0,17.0,0.0,0.0,20819.0,58.0,38199.0,0.0,73195,20877.0,14119.0,19.289569,1.802036,17.357743
Inversiones Bitec S A S,0.0,0.0,8160.0,310.0,3158.0,111.0,0.0,0.0,13362.0,27.0,62092.0,0.0,87220,13389.0,11739.0,13.459069,3.620729,9.355652
One Movil Sas,0.0,0.0,6738.0,476.0,2078.0,128.0,8.0,0.0,9664.0,52.0,76048.0,23.0,95215,9716.0,9420.0,9.893399,2.182429,7.076616


In [37]:
churn_inactividad=consultor_movimiento.sort_values('CHURN INACTIVIDAD',ascending=False)
churn_inactividad.head()

,ACTIVACION NO GROSS,CHURN FRAUDE,CHURN INACTIVIDAD,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,DESCONEXION NO GROSS,GROSS ADDS MIGRACION,GROSS ADDS NUEVA,GROSS ADDS PORTACION,PERMANECE,PERMANECE NO GROSS,TOTAL_CONSULTOR,GROSS_TOTAL,CHURN_TOTAL,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
CONSULTOR_NOMBRE_NIVEL_8,,,,,,,,,,,,,,,,,,
Fuerza Comercial Multiservicios Sas,0.0,0.0,12705.0,78.0,1319.0,17.0,0.0,0.0,20819.0,58.0,38199.0,0.0,73195,20877.0,14119.0,19.289569,1.802036,17.357743
Aloha Comunicaciones Sas,0.0,0.0,12431.0,192.0,14415.0,75.0,39.0,1.0,11284.0,23.0,63205.0,25.0,101690,11308.0,27113.0,26.662405,14.175435,12.224408
Tech Red Sas,0.0,0.0,12218.0,440.0,1377.0,152.0,61.0,0.0,26298.0,87.0,131343.0,7.0,171983,26385.0,14187.0,8.249071,0.800661,7.104191
Servicomunicaciones Caribe B C S A S,0.0,0.0,8368.0,68.0,542.0,26.0,2.0,0.0,8298.0,8.0,25697.0,6.0,43015,8306.0,9004.0,20.932233,1.260026,19.453679
Inversiones Bitec S A S,0.0,0.0,8160.0,310.0,3158.0,111.0,0.0,0.0,13362.0,27.0,62092.0,0.0,87220,13389.0,11739.0,13.459069,3.620729,9.355652


In [39]:
churn_portacion=consultor_movimiento.sort_values('CHURN PORTACION',ascending=False)
churn_portacion.head()

,ACTIVACION NO GROSS,CHURN FRAUDE,CHURN INACTIVIDAD,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,DESCONEXION NO GROSS,GROSS ADDS MIGRACION,GROSS ADDS NUEVA,GROSS ADDS PORTACION,PERMANECE,PERMANECE NO GROSS,TOTAL_CONSULTOR,GROSS_TOTAL,CHURN_TOTAL,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
CONSULTOR_NOMBRE_NIVEL_8,,,,,,,,,,,,,,,,,,
Aloha Comunicaciones Sas,0.0,0.0,12431.0,192.0,14415.0,75.0,39.0,1.0,11284.0,23.0,63205.0,25.0,101690,11308.0,27113.0,26.662405,14.175435,12.224408
Inversiones Bitec S A S,0.0,0.0,8160.0,310.0,3158.0,111.0,0.0,0.0,13362.0,27.0,62092.0,0.0,87220,13389.0,11739.0,13.459069,3.620729,9.355652
Ventas 577 Ltda,0.0,0.0,4755.0,296.0,3150.0,88.0,2.0,0.0,3503.0,11.0,43989.0,3.0,55797,3514.0,8289.0,14.855637,5.645465,8.521964
One Movil Sas,0.0,0.0,6738.0,476.0,2078.0,128.0,8.0,0.0,9664.0,52.0,76048.0,23.0,95215,9716.0,9420.0,9.893399,2.182429,7.076616
Conecta Telecomunciaciones Sas,0.0,0.0,7021.0,88.0,1993.0,21.0,0.0,0.0,6960.0,9.0,20233.0,1.0,36326,6969.0,9123.0,25.114243,5.486428,19.327754


In [40]:
gross_total=consultor_movimiento.sort_values('GROSS_TOTAL',ascending=False)
gross_total.head()

,ACTIVACION NO GROSS,CHURN FRAUDE,CHURN INACTIVIDAD,CHURN MIGRACION,CHURN PORTACION,CHURN VOLUNTARIO,DESCONEXION NO GROSS,GROSS ADDS MIGRACION,GROSS ADDS NUEVA,GROSS ADDS PORTACION,PERMANECE,PERMANECE NO GROSS,TOTAL_CONSULTOR,GROSS_TOTAL,CHURN_TOTAL,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
CONSULTOR_NOMBRE_NIVEL_8,,,,,,,,,,,,,,,,,,
Tech Red Sas,0.0,0.0,12218.0,440.0,1377.0,152.0,61.0,0.0,26298.0,87.0,131343.0,7.0,171983,26385.0,14187.0,8.249071,0.800661,7.104191
Fuerza Comercial Multiservicios Sas,0.0,0.0,12705.0,78.0,1319.0,17.0,0.0,0.0,20819.0,58.0,38199.0,0.0,73195,20877.0,14119.0,19.289569,1.802036,17.357743
Inversiones Bitec S A S,0.0,0.0,8160.0,310.0,3158.0,111.0,0.0,0.0,13362.0,27.0,62092.0,0.0,87220,13389.0,11739.0,13.459069,3.620729,9.355652
Conexxion Outsourcing sas,0.0,0.0,7001.0,12.0,16.0,2.0,0.0,0.0,13029.0,10.0,16207.0,0.0,36277,13039.0,7031.0,19.381426,0.044105,19.298729
Aloha Comunicaciones Sas,0.0,0.0,12431.0,192.0,14415.0,75.0,39.0,1.0,11284.0,23.0,63205.0,25.0,101690,11308.0,27113.0,26.662405,14.175435,12.224408


In [9]:
#DATOS DEL ULTIMO MES de como fue el CHURN para agregar a la 3ra diapositiva
query2=f'''
SELECT  MOVIMIENTO_TIPO_NOMBRE,
        COUNT(SUBSCRIBER_ID) AS TOTAL
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   PERIODO_PROCESO_CODIGO=202210
        AND SERVICIO='Prepaid'
        
GROUP BY MOVIMIENTO_TIPO_NOMBRE
'''
movimientos_agosto=cst.consulta(query2)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [10]:
movimientos_agosto

,MOVIMIENTO_TIPO_NOMBRE,TOTAL
0,GROSS ADDS PORTACION,4996
1,CHURN VOLUNTARIO,1758
2,CHURN FRAUDE,1
3,ACTIVACION NO GROSS,11999
4,GROSS ADDS MIGRACION,4422
5,PERMANECE NO GROSS,21894
6,CHURN MIGRACION,5930
7,CHURN PORTACION,41957
8,GROSS ADDS NUEVA,210318
9,CHURN INACTIVIDAD,155442


In [11]:
movimientos_agosto['TOTAL'].sum()

1555509

In [13]:
consultores_historico

,CONSULTOR_NOMBRE_NIVEL_8,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT
0,Joan Julian Arias Giraldo,12228,6581,3482,126,5655,11,0
1,Digital Inspire SAS,25189,16750,10253,26,8488,7,0
2,Aloha Comunicaciones Sas,282058,206946,70782,129745,63961,11310,0
3,Jorge Armando Arcos Ortiz,6200,1823,874,11,3247,1024,0
4,Rodrigo Alexander Barrero Fernandez,2724,1470,522,300,1262,2,0
...,...,...,...,...,...,...,...,...
724,Grupo Octa SAS,115,98,81,0,17,0,0
725,Maria Carolina Rodriguez Pineda,1316,709,308,66,606,0,0
726,MyF Soluciones Integrales Occidente,14089,1178,108,993,8166,4554,0
727,Harwin Castiblanco Sepulveda,595,31,17,0,276,280,0


In [15]:
consultores_historico["RATIO_CHURN"]=consultores_historico.apply(lambda x:division(x["CHURN_TOTAL"],x["GROSS_TOTAL"]),axis=1)
consultores_historico["RATIO_CHURN_PORTACION"]=consultores_historico.apply(lambda x:division(x["CHURN_PORTACION"],x["GROSS_TOTAL"]),axis=1)
consultores_historico["RATIO_CHURN_INACTIVIDAD"]=consultores_historico.apply(lambda x:division(x["CHURN_INACTIVIDAD"],x["GROSS_TOTAL"]),axis=1)
consultores_historico

,CONSULTOR_NOMBRE_NIVEL_8,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
0,Joan Julian Arias Giraldo,12228,6581,3482,126,5655,11,0,53.819104,28.475630,1.030422
1,Digital Inspire SAS,25189,16750,10253,26,8488,7,0,66.497281,40.704276,0.103220
2,Aloha Comunicaciones Sas,282058,206946,70782,129745,63961,11310,0,73.370016,25.094839,45.999404
3,Jorge Armando Arcos Ortiz,6200,1823,874,11,3247,1024,0,29.403226,14.096774,0.177419
4,Rodrigo Alexander Barrero Fernandez,2724,1470,522,300,1262,2,0,53.964758,19.162996,11.013216
...,...,...,...,...,...,...,...,...,...,...,...
724,Grupo Octa SAS,115,98,81,0,17,0,0,85.217391,70.434783,0.000000
725,Maria Carolina Rodriguez Pineda,1316,709,308,66,606,0,0,53.875380,23.404255,5.015198
726,MyF Soluciones Integrales Occidente,14089,1178,108,993,8166,4554,0,8.361133,0.766555,7.048052
727,Harwin Castiblanco Sepulveda,595,31,17,0,276,280,0,5.210084,2.857143,0.000000


In [16]:
query=f'''
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        B.CONSULTOR_NOMBRE_NIVEL_8

FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B 
ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE   A.PERIODO_PROCESO_CODIGO=202210
        AND A.SERVICIO='Prepaid'
'''
ending_agosto=cst.consulta(query)

c:\Users\HP EliteBook 840 G3\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [19]:
resultado_dealers=ending_agosto.merge(consultores_historico,right_on='CONSULTOR_NOMBRE_NIVEL_8',left_on='CONSULTOR_NOMBRE_NIVEL_8')

In [24]:
base=resultado_dealers.groupby(['CONSULTOR_NOMBRE_NIVEL_8'])[['SUBSCRIBER_ID']].count().reset_index().sort_values('SUBSCRIBER_ID',ascending=False)
base['PARTICIPACION']=(base['SUBSCRIBER_ID']/base['SUBSCRIBER_ID'].sum())*100
base.head()

,CONSULTOR_NOMBRE_NIVEL_8,SUBSCRIBER_ID,PARTICIPACION
626,Tech Red Sas,171983,11.056381
33,Aloha Comunicaciones Sas,101690,6.537410
538,One Movil Sas,95215,6.121147
301,Inversiones Bitec S A S,87220,5.607168
202,Dsurtiendo Sas,75129,4.829866


In [26]:
base.merge(consultores_historico,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8')

,CONSULTOR_NOMBRE_NIVEL_8,SUBSCRIBER_ID,PARTICIPACION,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
0,Tech Red Sas,171983,11.056381,404267,230070,49203,148369,145462,27877,0,56.910408,12.170917,36.700745
1,Aloha Comunicaciones Sas,101690,6.537410,282058,206946,70782,129745,63961,11310,0,73.370016,25.094839,45.999404
2,One Movil Sas,95215,6.121147,203930,117619,19562,91662,76048,9720,0,57.676163,9.592507,44.947776
3,Inversiones Bitec S A S,87220,5.607168,177851,102349,26787,72124,62092,13389,0,57.547610,15.061484,40.553047
4,Dsurtiendo Sas,75129,4.829866,137954,68521,17333,48361,57130,11052,0,49.669455,12.564333,35.055888
...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,Sertek Sas,1,0.000064,511,501,5,493,10,0,0,98.043053,0.978474,96.477495
685,Diego Ivan Gutierrez Garc,1,0.000064,3,2,2,0,1,0,0,66.666667,66.666667,0.000000
686,Federacion Nacional De Comerciantes Fena,1,0.000064,55,43,34,0,12,0,0,78.181818,61.818182,0.000000
687,ICCS COLOMBIA SAS,1,0.000064,3651,535,453,0,1971,1011,0,14.653520,12.407560,0.000000


In [39]:
permanece1=resultado_dealers['MOVIMIENTO_TIPO_NOMBRE']=='PERMANECE'
permanece=resultado_dealers[permanece1]
permanece=permanece.groupby(['CONSULTOR_NOMBRE_NIVEL_8'])[['SUBSCRIBER_ID']].count().reset_index().sort_values('SUBSCRIBER_ID',ascending=False)
permanece['PARTICIPACION']=(permanece['SUBSCRIBER_ID']/permanece['SUBSCRIBER_ID'].sum())*100
permanece.head()
permanece.merge(consultores_historico,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8')

,CONSULTOR_NOMBRE_NIVEL_8,SUBSCRIBER_ID,PARTICIPACION,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
0,Tech Red Sas,131343,11.998674,404267,230070,49203,148369,145462,27877,0,56.910408,12.170917,36.700745
1,One Movil Sas,76048,6.947269,203930,117619,19562,91662,76048,9720,0,57.676163,9.592507,44.947776
2,Aloha Comunicaciones Sas,63205,5.774013,282058,206946,70782,129745,63961,11310,0,73.370016,25.094839,45.999404
3,Inversiones Bitec S A S,62092,5.672336,177851,102349,26787,72124,62092,13389,0,57.547610,15.061484,40.553047
4,Dsurtiendo Sas,57130,5.219039,137954,68521,17333,48361,57130,11052,0,49.669455,12.564333,35.055888
...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,Victoria Eugenia Gonzalez Ramirez,1,0.000091,99,33,18,0,66,0,0,33.333333,18.181818,0.000000
650,Jorge Enrique Hernandez Cortes,1,0.000091,175,8,6,0,77,86,0,4.571429,3.428571,0.000000
651,CORPORACION SOCIAL Y CULTURAL PAZOSFERA,1,0.000091,2,1,0,1,1,0,0,50.000000,0.000000,50.000000
652,Javier Mauricio Aparicio Jaimes,1,0.000091,205,16,16,0,70,108,0,7.804878,7.804878,0.000000


In [48]:
churn_total=resultado_dealers[resultado_dealers['MOVIMIENTO_TIPO_NOMBRE'].isin(['CHURN PORTACION','CHURN INACTIVIDAD','CHURN FRAUDE','CHURN MIGRACION','CHURN VOLUNTARIO'])]
churn_total=churn_total.groupby(['CONSULTOR_NOMBRE_NIVEL_8'])[['SUBSCRIBER_ID']].count().reset_index().sort_values('SUBSCRIBER_ID',ascending=False)
churn_total['PARTICIPACION']=(churn_total['SUBSCRIBER_ID']/churn_total['SUBSCRIBER_ID'].sum())*100
churn_total.head()
churn_total.merge(consultores_historico,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8')

,CONSULTOR_NOMBRE_NIVEL_8,SUBSCRIBER_ID,PARTICIPACION,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
0,Aloha Comunicaciones Sas,27113,13.220179,282058,206946,70782,129745,63961,11310,0,73.370016,25.094839,45.999404
1,Tech Red Sas,14187,6.917518,404267,230070,49203,148369,145462,27877,0,56.910408,12.170917,36.700745
2,Fuerza Comercial Multiservicios Sas,14119,6.884362,113665,46777,11959,26487,45032,21610,0,41.153389,10.521269,23.302688
3,Inversiones Bitec S A S,11739,5.723884,177851,102349,26787,72124,62092,13389,0,57.547610,15.061484,40.553047
4,One Movil Sas,9420,4.593150,203930,117619,19562,91662,76048,9720,0,57.676163,9.592507,44.947776
...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,Sarah Isabel Vergara Muño,1,0.000488,505,402,155,20,104,0,0,79.603960,30.693069,3.960396
542,Kimberly Escobar Giraldo,1,0.000488,242,18,16,0,79,146,0,7.438017,6.611570,0.000000
543,Santiago Alonso Ospina Moreno,1,0.000488,1054,244,100,3,549,250,0,23.149905,9.487666,0.284630
544,Daniela Andrea Mazo Baena,1,0.000488,552,70,25,1,370,104,0,12.681159,4.528986,0.181159


In [49]:
churn_inactividad=resultado_dealers[resultado_dealers['MOVIMIENTO_TIPO_NOMBRE'].isin(['CHURN INACTIVIDAD'])]
churn_inactividad=churn_inactividad.groupby(['CONSULTOR_NOMBRE_NIVEL_8'])[['SUBSCRIBER_ID']].count().reset_index().sort_values('SUBSCRIBER_ID',ascending=False)
churn_inactividad['PARTICIPACION']=(churn_inactividad['SUBSCRIBER_ID']/churn_inactividad['SUBSCRIBER_ID'].sum())*100
churn_inactividad.head()
churn_inactividad.merge(consultores_historico,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8')

,CONSULTOR_NOMBRE_NIVEL_8,SUBSCRIBER_ID,PARTICIPACION,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
0,Fuerza Comercial Multiservicios Sas,12705,8.173467,113665,46777,11959,26487,45032,21610,0,41.153389,10.521269,23.302688
1,Aloha Comunicaciones Sas,12431,7.997195,282058,206946,70782,129745,63961,11310,0,73.370016,25.094839,45.999404
2,Tech Red Sas,12218,7.860166,404267,230070,49203,148369,145462,27877,0,56.910408,12.170917,36.700745
3,Servicomunicaciones Caribe B C S A S,8368,5.383358,79161,44080,3805,34994,26160,8334,0,55.683986,4.806660,44.206112
4,Inversiones Bitec S A S,8160,5.249546,177851,102349,26787,72124,62092,13389,0,57.547610,15.061484,40.553047
...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Vanegas Frias Leydis Dayling,1,0.000643,540,382,152,25,158,0,0,70.740741,28.148148,4.629630
470,Dairo Amauris Cañas De La Rosa,1,0.000643,3677,1399,349,1,1889,359,0,38.047321,9.491433,0.027196
471,Freddy Alexander Tibaquira Puentes,1,0.000643,2612,1644,792,48,973,1,0,62.940276,30.321593,1.837672
472,Victor Alfonso Preciado Ulloa,1,0.000643,4414,2228,1090,18,1934,239,0,50.475759,24.694155,0.407793


In [50]:
churn_portacion=resultado_dealers[resultado_dealers['MOVIMIENTO_TIPO_NOMBRE'].isin(['CHURN PORTACION'])]
churn_portacion=churn_portacion.groupby(['CONSULTOR_NOMBRE_NIVEL_8'])[['SUBSCRIBER_ID']].count().reset_index().sort_values('SUBSCRIBER_ID',ascending=False)
churn_portacion['PARTICIPACION']=(churn_portacion['SUBSCRIBER_ID']/churn_portacion['SUBSCRIBER_ID'].sum())*100
churn_portacion.head()
churn_portacion.merge(consultores_historico,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8')

,CONSULTOR_NOMBRE_NIVEL_8,SUBSCRIBER_ID,PARTICIPACION,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
0,Aloha Comunicaciones Sas,14415,34.356603,282058,206946,70782,129745,63961,11310,0,73.370016,25.094839,45.999404
1,Inversiones Bitec S A S,3158,7.526754,177851,102349,26787,72124,62092,13389,0,57.547610,15.061484,40.553047
2,Ventas 577 Ltda,3150,7.507686,188224,140476,65239,69957,43990,3515,0,74.632353,34.660298,37.166886
3,One Movil Sas,2078,4.952690,203930,117619,19562,91662,76048,9720,0,57.676163,9.592507,44.947776
4,Conecta Telecomunciaciones Sas,1993,4.750101,106427,71329,12191,42495,26792,7618,0,67.021526,11.454800,39.928777
...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,Carlos Felipe Cossio Herrera,1,0.002383,289,24,14,0,152,108,0,8.304498,4.844291,0.000000
418,Manuel Orlando Fonseca Gamboa,1,0.002383,297,29,14,0,259,6,0,9.764310,4.713805,0.000000
419,Luis Alejandro Samudio Armesto,1,0.002383,473,82,37,1,394,0,0,17.336152,7.822410,0.211416
420,Hector Fabio Acosta Luna,1,0.002383,323,187,64,4,137,0,0,57.894737,19.814241,1.238390


In [51]:
gross=resultado_dealers[resultado_dealers['MOVIMIENTO_TIPO_NOMBRE'].isin(['GROSS ADDS NUEVA','GROSS ADDS PORTACION','GROSS ADDS MIGRACION'])]
gross=gross.groupby(['CONSULTOR_NOMBRE_NIVEL_8'])[['SUBSCRIBER_ID']].count().reset_index().sort_values('SUBSCRIBER_ID',ascending=False)
gross['PARTICIPACION']=(gross['SUBSCRIBER_ID']/gross['SUBSCRIBER_ID'].sum())*100
gross.head()
gross.merge(consultores_historico,left_on='CONSULTOR_NOMBRE_NIVEL_8',right_on='CONSULTOR_NOMBRE_NIVEL_8')

,CONSULTOR_NOMBRE_NIVEL_8,SUBSCRIBER_ID,PARTICIPACION,GROSS_TOTAL,CHURN_TOTAL,CHURN_PORTACION,CHURN_INACTIVIDAD,PERMANECE_ACT,GROSS_ACT,CHURN_ACT,RATIO_CHURN,RATIO_CHURN_PORTACION,RATIO_CHURN_INACTIVIDAD
0,Tech Red Sas,26385,12.007591,404267,230070,49203,148369,145462,27877,0,56.910408,12.170917,36.700745
1,Fuerza Comercial Multiservicios Sas,20877,9.500947,113665,46777,11959,26487,45032,21610,0,41.153389,10.521269,23.302688
2,Inversiones Bitec S A S,13389,6.093221,177851,102349,26787,72124,62092,13389,0,57.547610,15.061484,40.553047
3,Conexxion Outsourcing sas,13039,5.933939,48816,14631,3906,10464,19397,15075,0,29.971731,8.001475,21.435595
4,Aloha Comunicaciones Sas,11308,5.146175,282058,206946,70782,129745,63961,11310,0,73.370016,25.094839,45.999404
...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,Gilma Ponguta Lopez,1,0.000455,4863,2731,1332,517,2140,1,0,56.158750,27.390500,10.631298
544,Yamile Cuellar Prieto,1,0.000455,4411,2738,1374,402,1682,1,0,62.072092,31.149399,9.113580
545,Angela Lorena Durango Rol,1,0.000455,3264,2291,955,220,976,1,0,70.189951,29.258578,6.740196
546,Gisella Trochez Muñoz,1,0.000455,826,211,81,19,605,13,0,25.544794,9.806295,2.300242


In [ ]:
#Query para ver el churn portacion por la antiguedad de los usuarios, preguntar regla de 1 mes
query=f'''
SELECT RANGO_ANTIGUEDAD,COUNT(SUBSCRIBER_ID) as TOTAL
FROM (
SELECT  SUBSCRIBER_ID, 
        FECHA_BAJA-FECHA_ALTA AS ANTIGUEDAD,
        CASE 
            WHEN FECHA_BAJA-FECHA_ALTA BETWEEN 30 AND 60 THEN '2 MESES'
            WHEN FECHA_BAJA-FECHA_ALTA BETWEEN 61 AND 90 THEN '3 MESES'
            WHEN FECHA_BAJA-FECHA_ALTA >91 THEN 'MAYOR A 3 MESES'
            ELSE '1 MES'
        END AS RANGO_ANTIGUEDAD
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   PERIODO_PROCESO_CODIGO=202208
        AND SERVICIO='Prepaid'
        AND MOVIMIENTO_TIPO_NOMBRE ='CHURN PORTACION' ) B
GROUP BY RANGO_ANTIGUEDAD
'''

In [ ]:
#Query de churn portacion por operador porta origen
query=f'''
SELECT OPERADOR_PORTA_ORIGEN,COUNT(SUBSCRIBER_ID) AS TOTAL
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
WHERE   PERIODO_PROCESO_CODIGO=202208
        AND SERVICIO='Prepaid'
        AND MOVIMIENTO_TIPO_NOMBRE ='CHURN PORTACION'
GROUP BY OPERADOR_PORTA_ORIGEN 
'''

In [ ]:
#Query de churn portacion por operador porta DESTINO
query=f'''
SELECT OPERADOR_PORTA_DESTINO,COUNT(SUBSCRIBER_ID) AS TOTAL
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
WHERE   PERIODO_PROCESO_CODIGO=202208
        AND SERVICIO='Prepaid'
        AND MOVIMIENTO_TIPO_NOMBRE ='CHURN PORTACION'
GROUP BY OPERADOR_PORTA_DESTINO
'''

In [ ]:
#query para los rangos de dias quitar los DD y el 3er filtro para los meses 
query=f'''
SELECT TO_NUMBER(TO_CHAR(FECHA_ALTA,'YYYYMMDD')),COUNT(SUBSCRIBER_ID)
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING
WHERE   PERIODO_PROCESO_CODIGO=202208
        AND SERVICIO='Prepaid'
        AND MOVIMIENTO_TIPO_NOMBRE ='CHURN PORTACION'
        AND TO_NUMBER(TO_CHAR(FECHA_ALTA,'YYYYMM'))=202207
GROUP BY TO_NUMBER(TO_CHAR(FECHA_ALTA,'YYYYMMDD'))
ORDER BY TO_NUMBER(TO_CHAR(FECHA_ALTA,'YYYYMMDD')) DESC


'''

In [ ]:
#INACTIVIDADES ACUMULADAS query cambiar las temporalidades
query=f'''
WITH
    BASE AS(
        SELECT /*+PARALLEL(8)*/
            SUBSCRIBER_ID
        FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
        WHERE SERVICIO = 'Prepaid'
            AND PERIODO_PROCESO_CODIGO = 202208
            AND MOVIMIENTO_NOMBRE IN ('PERMANECE', 'ACTIVACION')
            AND TO_NUMBER(TO_CHAR(FECHA_ALTA, 'YYYYMM')) = 202205
    ),
    
    DATOS AS(
        SELECT /*+PARALLEL(8)*/
            SUBSCRIBER_ID,
            MAX(PERIODO_PROCESO_CODIGO) AS ULT_DATOS
        FROM AGG_DL_DATA
        WHERE PERIODO_PROCESO_CODIGO BETWEEN 20220601 AND 20220831
        GROUP BY SUBSCRIBER_ID
    ),
    
    VOZ AS(
        SELECT /*+PARALLEL(8)*/
            SUBSCRIBER_ID,
            MAX(PERIODO_PROCESO_CODIGO) AS ULT_VOZ
        FROM AGG_DL_VOICE
        WHERE PERIODO_PROCESO_CODIGO BETWEEN 20220601 AND 20220831
        GROUP BY SUBSCRIBER_ID
    )



   
    SELECT /*+PARALLEL(8)*/
        A.SUBSCRIBER_ID,
        CASE
            WHEN COALESCE(B.ULT_DATOS, -2) >= C.ULT_VOZ THEN B.ULT_DATOS
            WHEN COALESCE(C.ULT_VOZ, -2) >= B.ULT_DATOS THEN B.ULT_DATOS
        END ULT_TRAFICO
    FROM BASE A
    LEFT JOIN DATOS B ON A.SUBSCRIBER_ID = B.SUBSCRIBER_ID
    LEFT JOIN VOZ C ON A.SUBSCRIBER_ID = C.SUBSCRIBER_ID
'''

In [ ]:
#query de tipo de usuario 
query=f'''
SELECT
    SUBSCRIBER_ID,
    TIPO_CEDULA,
    ROUND(BYTES_TOTALES/1024/1024) AS MB_TOTALES,
    BILLETERA_GRATUITA,
    MIN_TOTALES_SALIENTES,
    CASE
        WHEN (COALESCE(BYTES_TOTALES/1024/1024, 0) <= 1 OR COALESCE(MIN_TOTALES_SALIENTES, 0) <= 1) AND TIPO_CEDULA = 'CEDULAS_FAKE' THEN 'USUARIO FAKE'
        WHEN (COALESCE(BYTES_TOTALES/1024/1024, 0) <= 1 AND COALESCE(MIN_TOTALES_SALIENTES, 0) <= 1) AND TIPO_CEDULA = 'CEDULAS_NORMAL' THEN 'USUARIO FAKE TRAFICO'
        WHEN (COALESCE(BYTES_TOTALES/1024/1024, 0) > 1 AND COALESCE(MIN_TOTALES_SALIENTES, 0) > 1) AND COALESCE(BILLETERA_GRATUITA, 0) > 20 THEN 'USUARIO FAKE FRAUDE'
        ELSE 'USUARIO NORMAL'
    END TIPO_USUARIO
FROM CR_ML_PRE_USUARIOS_INSUMOS
'''

In [ ]:
#QUERY QUE da la inactividad por regional 
query=f'''
SELECT  CONSULTOR_REGIONAL_NOMBRE,COUNT(SUBSCRIBER_ID) AS TOTAL 
FROM (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        B.CONSULTOR_REGIONAL_NOMBRE

FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
LEFT JOIN DWH_BODEGA_WOM.DIM_CONSULTORES B 
ON A.CONSULTOR_DK=B.CONSULTOR_DK
WHERE   A.PERIODO_PROCESO_CODIGO=202208
        AND A.SERVICIO='Prepaid'
        AND A.MOVIMIENTO_TIPO_NOMBRE='CHURN INACTIVIDAD' ) C
GROUP BY CONSULTOR_REGIONAL_NOMBRE
'''

In [ ]:
####################

In [ ]:
#ORGANIZAR CHURN PORTACION POR OEPRADOR ORIGEN 
#base
query=f'''
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        A.OPERADOR_PORTA_ORIGEN,
        A.OPERADOR_PORTA_DESTINO
        
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   A.PERIODO_PROCESO_CODIGO=202210
        AND A.MOVIMIENTO_TIPO_NOMBRE='CHURN PORTACION'
        AND A.SERVICIO='Prepaid'
'''

In [ ]:
# QUERY QUE ORGANIZA EL CHURN PORTACION POR OPERADOR PORTA ORIGEN
query=f'''
SELECT OPERADOR_PORTA_ORIGEN,COUNT(*)
FROM (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        A.OPERADOR_PORTA_ORIGEN,
        A.OPERADOR_PORTA_DESTINO
        
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   A.PERIODO_PROCESO_CODIGO=202206
        AND A.MOVIMIENTO_TIPO_NOMBRE='CHURN PORTACION'
        AND A.SERVICIO='Prepaid'
)
GROUP BY OPERADOR_PORTA_ORIGEN
'''

In [ ]:
#query que organiza el churn portacion por operador origen a destino 
query=f'''
SELECT OPERADOR_PORTA_ORIGEN,OPERADOR_PORTA_DESTINO,COUNT(*)
FROM (
SELECT  A.SUBSCRIBER_ID,
        A.MOVIMIENTO_TIPO_NOMBRE,
        A.OPERADOR_PORTA_ORIGEN,
        A.OPERADOR_PORTA_DESTINO
        
FROM DWH_BODEGA_WOM.FCT_SUBSCRIBERS_ENDING A
WHERE   A.PERIODO_PROCESO_CODIGO=202206
        AND A.MOVIMIENTO_TIPO_NOMBRE='CHURN PORTACION'
        AND A.SERVICIO='Prepaid'
)
GROUP BY OPERADOR_PORTA_ORIGEN,OPERADOR_PORTA_DESTINO
ORDER BY OPERADOR_PORTA_ORIGEN,OPERADOR_PORTA_DESTINO 
'''